In [1]:
from collections import Counter
import os
import pickle
import math

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


base_directory = os.path.abspath(os.curdir)
data_directory = os.path.join(base_directory, "data")

In [25]:
dfx = pd.read_pickle(os.path.join(data_directory,'viewing.pkl'))
# dfx['Program Genre'].unique()
x_mat = dfx.as_matrix(dfx.columns)
dfx.columns


Index(['Device ID', 'Program Code', 'view_duration', 'Program Title',
       'Program Genre', 'Duration', 'completion_precent', 'df_id', 'prev_1',
       'prev_2', 'last_in_seq', 'event_weekday', 'part_of_day',
       'duration_bins', 'view_bins', 'completion_bins', 'prev_1_genre',
       'prev_2_genre'],
      dtype='object')

In [12]:
#utils for later
map_cols =  {}

feature_position = {}
feature_counter = 0

map_cols = {col: ind for ind, col in enumerate(dfx.columns)}

for ind, col in enumerate(dfx.columns):
    map_cols[col] = ind

print(map_cols)
label_in_train = [] # from Rom
titles_labels = {}

"""
feature_type_map = {
    # time related features
     t_0: day of the week
     t_1: part of day (morning, noon, evening, night) in bucket of 6
     t_2: day of week and part of day
    # demographic related:
     from Dafna
    # genre related:
     g_0: current genre
     g_1: 2-gram
     g_2:  3-gram
    # program related:
      duration : p_0
      p_1: view duration
      p_2: program completion 
      
    # interactions:
     i_0: g_0 * t_0
     i_1: g_0 * t_1
     i_2: g_1 * t_0
     i_3: g_1 * t_1
     i_4: g_2 * t_0
     i_5: g_2 * t_1
     
 } 
"""

{'Device ID': 0, 'Program Code': 1, 'view_duration': 2, 'Program Title': 3, 'Program Genre': 4, 'Duration': 5, 'completion_precent': 6, 'df_id': 7, 'prev_1': 8, 'prev_2': 9, 'last_in': 10, 'last_in_seq': 11, 'event_weekday': 12, 'part_of_day': 13, 'duration_bins': 14, 'view_bins': 15, 'completion_bins': 16}


'\nfeature_type_map = {\n    # time related features\n     t_0: day of the week\n     t_1: part of day (morning, noon, evening, night) in bucket of 6\n     t_2: day of week and part of day\n    # demographic related:\n     from Dafna\n    # genre related:\n     g_0: current genre\n     g_1: 2-gram\n     g_2:  3-gram\n    # program related:\n      duration\n      code/title\n      \n    # interactions:\n     i_0: g_0 * t_0\n     i_1: g_0 * t_1\n     i_2: g_1 * t_0\n     i_3: g_1 * t_1\n     i_4: g_2 * t_0\n     i_5: g_2 * t_1\n     \n } \n'

In [13]:
# time features - no need to count
# t_0: day of the week
# t_1: part of day (morning, noon, evening, night) in bucket of 6

feature_candidates_list = []

def feature_from_dfx(colname, suffix, add):
    
    for value in dfx[colname].unique():
#         name = str(value) + suffix
        name = "{}_{}".format(value, suffix)
        
        if add:
            feature_position[name] = counter
            counter += 1
        else:
            feature_candidates_list.append(name)


# feature_type_map:

## time related features
 * t_0: day of the week
 * t_1: part of day (morning, noon, evening, night) in bucket of 6
 * t_2: day of week and part of day
 
## demographic related:
 * from Dafna
    
## genre related:
* g_0: current genre
* g_1: 2-gram
* g_2:  3-gram

## program related:
* duration : p_0
* p_1: view duration
* p_2: program completion 
      
## interactions:
* i_0: g_0 * t_0
* i_1: g_0 * t_1
* i_2: g_1 * t_0
* i_3: g_1 * t_1
* i_4: g_2 * t_0
* i_5: g_2 * t_1
   

In [31]:
from itertools import count
# features we need to consider anyways

pos = count()

col_action = {
    'Program Genre': ['counter', 'g0'],
    'event_weekday': ['unique', 't0'],
    'part_of_day': ['unique', 't1'],
    'duration_bins': ['unique', 'p0'],
    'view_bins': ['unique', 'p1'],
    'completion_bins': ['unique', 'p2'],
    ('part_of_day', 'event_weekday'): ['interact', 't3'],
    ('Program Genre', 'event_weekday'): ['interact', 'i0'],
    ('Program Genre', 'part_of_day'): ['interact', 'i1'],
    ('prev_1_genre', 'Program Genre'): ['interact', 'g1'],
    ('prev_2_genre' , 'prev_1_genre', 'Program Genre'): ['double_interact', 'g2']  
}

thrash_holds = {
    'Program Genre': 30,
    ('part_of_day', 'event_weekday'): 30,
    ('Program Genre', 'event_weekday'): 30,
    ('Program Genre', 'part_of_day'): 30,
    ('prev_1_genre', 'Program Genre'): 30,
    ('prev_2_genre' , 'prev_1_genre', 'Program Genre'): 30
    
}    

feature_id = dict()
for col, (action, prefix) in col_action.items():
    if action == 'counter':
        # todo: take it out to function
        a = dfx[col].fillna('Na').value_counts()
        feature_id.update({"{}_{}".format(prefix, feature_val): next(pos) 
                                          for feature_val in a[a > thrash_holds[col]].index})
    if action == 'unique':
        feature_id.update({"{}_{}".format(prefix, feature_val): 
                           next(pos) for feature_val in dfx[col].astype(str).unique()})
    if action == 'interact':
        col_1, col_2 = col
        a = dfx.groupby([col_1, col_2], as_index=True).size() #.reset_index()
        a = a[a > thrash_holds[col]].reset_index()
        feature_id.update({"{}_{}_{}".format(prefix, col1, col2): next(pos) for col1,col2,val in a.values})
    if action == 'double_interact':
        col_1, col_2, col_3 = col
        a = dfx.groupby([col_1, col_2, col_3], as_index=True).size() #.reset_index()
        a = a[a > thrash_holds[col]].reset_index()
        feature_id.update({"{}_{}_{}_{}".format(prefix, col1, col2, col3): next(pos) for col1, col2, col3, val in a.values})

feature_id
    
# # Counter(dfx['Program Genre'].values)
# names_dict = dict()
# for a_i in dfx['duration_bins'].cat.categories:
#     names_dict.update({str(a_i): "{}_{}".format(a_i.left,a_i.right)})
# #     print(a_i.left,a_i.right)
   
# names_dict

{'g0_Football,Playoff-sports,Sports-event': 0,
 'g0_News': 1,
 'g0_Sitcom': 2,
 'g0_Consumer': 3,
 'g0_Football,Sports-non-event': 4,
 'g0_Reality': 5,
 'g0_Na': 6,
 'g0_Crime-drama': 7,
 'g0_Comedy': 8,
 'g0_Drama': 9,
 'g0_Shopping': 10,
 'g0_Crime,Reality': 11,
 'g0_House/garden,Reality': 12,
 'g0_Animated,Sitcom': 13,
 'g0_Children,Sitcom': 14,
 'g0_Basketball,Sports-event': 15,
 'g0_Newsmagazine': 16,
 'g0_News,Sports-non-event,Sports-talk': 17,
 'g0_Religious': 18,
 'g0_Action': 19,
 'g0_Cooking,Reality': 20,
 'g0_News,Talk': 21,
 'g0_Football,Sports-non-event,Sports-talk': 22,
 'g0_Weather': 23,
 'g0_News,Politics,Public-affairs,Talk': 24,
 'g0_Action,Suspense': 25,
 'g0_Talk': 26,
 'g0_Documentary': 27,
 'g0_Drama,Politics': 28,
 'g0_Romance-comedy': 29,
 'g0_Sports-non-event': 30,
 'g0_Action,Crime-drama,Mystery,Suspense': 31,
 'g0_Game-show': 32,
 'g0_Documentary,Reality': 33,
 'g0_Drama,Law': 34,
 'g0_Suspense': 35,
 'g0_Auction,Collectibles,Reality': 36,
 'g0_Adventure,Docu

In [29]:
def get_prev_genres(idx, depth=2): 
    if np.isnan(idx):
        return np.nan
    if idx < 0 or idx > 2:
        raise ValueError("{} is not a valid depth".format(idx))
    label = [dfx.at[int(idx), 'Program Genre']]
    if idx > 0:
        label.append(dfx.at[int(idx), 'Program Genre'])
    return 

get_prev_genres(dfx.loc[0,'prev_1']) 

nan

In [29]:
# %%timeit -o -n 500
col_1, col_2, col_3 = ('prev_2_genre' , 'prev_1_genre', 'Program Genre')

a = dfx.groupby([col_1, col_2, col_3], as_index=True).size() #.reset_index()
a = a[a > thrash_holds[col]].reset_index()
{"{}_{}_{}_{}".format(prefix, col1, col2, col3): next(pos) for col1, col2, col3, val in a.values}
# a[a["prev_2_genre"] == '**']

{'g1_*_Comedy_Special': 579,
 'g1_**_*_Comedy': 580,
 'g1_**_*_Comedy,Crime-drama': 581,
 'g1_**_*_Crime-drama': 582,
 'g1_**_*_Na': 583,
 'g1_Action_Football,Playoff-sports,Sports-event_Action': 584,
 'g1_Animated,Sitcom_Animated,Sitcom_Animated,Sitcom': 585,
 'g1_Animated,Sitcom_Animated,Sitcom_News': 586,
 'g1_Auction,Collectibles,Reality_Auction,Collectibles,Reality_Auction,Collectibles,Reality': 587,
 'g1_Basketball,Sports-event_Football,Playoff-sports,Sports-event_Basketball,Sports-event': 588,
 'g1_Children,Sitcom_Children,Sitcom_Children,Sitcom': 589,
 'g1_Comedy_Comedy_Comedy': 590,
 'g1_Comedy_Entertainment,Talk_Entertainment,Talk': 591,
 'g1_Comedy_Special_Comedy': 592,
 'g1_Consumer_Consumer_Consumer': 593,
 'g1_Consumer_Consumer_Documentary,Special': 594,
 'g1_Consumer_Consumer_Shopping': 595,
 'g1_Consumer_Game-show_Consumer': 596,
 'g1_Consumer_Shopping_Consumer': 597,
 'g1_Cooking,Reality_Cooking,Reality_Cooking,Reality': 598,
 'g1_Cooking,Reality_News_News': 599,
 'g1_